# creating map from csv file 

1. data is [from](https://earthdata.nasa.gov/earth-observation-data/near-real-time/firms/viirs-i-band-active-fire-data)

1. Convert a mulitpart geometry into seprate geometries, the geometry is after subject to mapshaper

In [ ]:
import geopandas as gpd
# from shapely.geometry.polygon import Polygon
# from shapely.geometry.multipolygon import MultiPolygon

# def explode(gdf):
#     """ 
#     Explodes a geodataframe 
    
#     Will explode muti-part geometries into single geometries. Original index is
#     stored in column level_0 and zero-based count of geometries per multi-
#     geometry is stored in level_1
    
#     Args:
#         gdf (gpd.GeoDataFrame) : input geodataframe with multi-geometries
        
#     Returns:
#         gdf (gpd.GeoDataFrame) : exploded geodataframe with a new index 
#                                  and two new columns: level_0 and level_1
        
#     """
#     gs = gdf.explode()
#     gdf2 = gs.reset_index().rename(columns={0: 'geometry'})
#     gdf_out = gdf2.merge(gdf.drop('geometry', axis=1), left_on='level_0', right_index=True)
#     gdf_out = gdf_out.set_index(['level_0', 'level_1']).set_geometry('geometry')
#     gdf_out.crs = gdf.crs
#     return gdf_out

indata='../data/vector/India_states.json'
gdf=gpd.GeoDataFrame.from_file(indata)
# qq=explode(gdf)
gdf.to_file('India_states.shp', driver='ESRI Shapefile')
#gdf

In [ ]:
import matplotlib
from matplotlib.colors import Normalize
import matplotlib.cm as cm
from matplotlib.colors import LinearSegmentedColormap
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.pyplot as plt
plt.switch_backend('agg')
from matplotlib.collections import PatchCollection
from mpl_toolkits.basemap import Basemap
import numpy as np
import pandas as pd
from itertools import chain
from numpy import linspace
from shapely.geometry import Point, Polygon, MultiPoint, MultiPolygon
from descartes import PolygonPatch
import geopandas
from osgeo import gdal
from shapely.geometry import LineString
from pysal.esda.mapclassify import Natural_Breaks as nb
from pysal.esda.mapclassify import User_Defined as ud
import fiona
import os
from os.path import exists
import sys
import matplotlib.patheffects as pe
if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO


dba = pd.read_csv('../data/vector/fire_archive_V1_2871.csv')



def lookup(s):
    """
    This is an extremely fast approach to datetime parsing.
    For large data, the same dates are often repeated. Rather than
    re-parse these, we store all unique dates, parse them, and
    use a lookup to convert all dates.
    """
    return s.map({date:pd.to_datetime(date) for date in s.unique()})

dba.loc[:,'date']=lookup(dba['acq_date'])

dba['monthday'] = dba['date'].dt.strftime('%j')
dba['fileformate'] = dba['date'].dt.strftime('%Y_%m_%d')
dba['year'] = dba['date'].dt.strftime('%Y')


yearlist=dba.drop_duplicates('year')
yearlist1=yearlist['year'].tolist()




###########################
# choose dpi

dpichos=300

###########################
#speicify the path location of the script file, in which folder the shape file and data csv file has to be placed

path="./"
os.chdir(path)

############################
#location of the legend to show, choose legendpos=left or right, comment the line otherwise

#left=fig.add_axes([0.12, 0.1, 0.04, 0.25]) 

#right=fig.add_axes([0.8, 0.1, 0.03, 0.3]) 
#[left,right,height, width]
#legendpos=left

############################
#the background color, chnage the string 'w' to get different colors

tbg='w'

###########################
#output file type, edit the outputtype into '.jpg' to get it in jpg  

outputtype='.png'

##########################
# resolution/size, edit size into small to get the ouptut in small size 

India=[8.6,8.6]

size=India

##########################
#get the exxtent of shape file, state shape file read and takes the extent values from it

shp = fiona.open('India_states.shp')
bds = shp.bounds
shp.close()
extra = 0.01
ll = (bds[0], bds[1])
ur = (bds[2], bds[3])
coords = list(chain(ll, ur))
w, h = coords[2] - coords[0], coords[3] - coords[1]



##########################
#does reading of shape file and make a pandas datframe and read csv file to attach it with shape file
for year in yearlist1:
    dba1=dba[dba['year']=='2017']
    daylist=dba1.drop_duplicates('monthday')
    daylist1=daylist['monthday'].tolist()
    for indef,houre in enumerate(daylist1[0:12]):
        df5e=dba1[dba1.monthday==houre]
        df5e['daye']=pd.to_datetime(df5e['acq_date'], format='%Y-%m-%d')
        df5e['day1'] = df5e['daye'].dt.strftime('%Y-%b-%d')
        timet=df5e.iloc[-1]['day1']
        date=df5e.iloc[0]['fileformate']
        newfilename='viirs_'+date
        fig = plt.figure()
        statepa = fig.add_subplot(111, facecolor=tbg, frame_on=False)
        amap = Basemap(epsg=24374,lon_0=19.17,lat_0=72.98,ellps = 'WGS84',llcrnrlon=coords[0],llcrnrlat=coords[1],urcrnrlon=coords[2]+0.8,urcrnrlat=coords[3],lat_ts=0,resolution='i',suppress_ticks=True, ax=statepa)
        amap.readshapefile('India_states','India_states',color='none',zorder=2)
        df_map = pd.DataFrame({'geometry': [Polygon(xy) for xy in amap.India_states],'censuscode': [state['censuscode'] for state in amap.India_states_info]})
        df_map['patches'] = df_map['geometry'].map(lambda x: PolygonPatch(x,fc='#d7dac2',ec='#ffffff', lw=.85, alpha=.9,zorder=4))
        ###READING THE FIRE DATA
        map_points = pd.Series([Point(amap(mapped_x, mapped_y)) for mapped_x, mapped_y in zip(df5e['longitude'], df5e['latitude'])])
        dev = amap.scatter([geom.x for geom in map_points],[geom.y for geom in map_points],1, marker='o', lw=0,facecolor='r', edgecolor='b',alpha=0.6, antialiased=True,label='Active fire locations', zorder=3)
        statepa.add_collection(PatchCollection(df_map['patches'].values, match_original=True))
        x1,y1=amap(85.0,31.0)
        plt.text(x1, y1, 'Himalayas', fontsize=12,fontweight='bold',ha='left',va='center',color='k')
        x1,y1=amap(67.0,16.0)
        plt.text(x1, y1, 'Arabian Sea', fontsize=12,fontweight='bold',ha='left',va='center',color='k')
        x1,y1=amap(85.2,13.0)
        plt.text(x1, y1, 'Bay of Bengal', fontsize=12,fontweight='bold',ha='left',va='center',color='k')
        x1,y1=amap(81.0,39.0)
        plt.text(x1, y1, 'Open Fires Detected on '+timet, fontsize=15,fontweight='bold',ha='center',va='center',color='k')
        citations='Source: NRT VIIRS 375 m \nActive Fire product VNP14IMGT. \nAvailable on-line [https://earthdata.nasa.gov/firms]. \nDOI:10.5067/FIRMS/VIIRS/VNP14IMGT.NRT.001.'
        x1,y1=amap(66.0,7.5)
        plt.text(x1, y1, citations, fontsize=6,fontweight='bold',ha='left',va='center',color='k')
        fig.set_size_inches(size[0],size[1])
        plt.savefig(newfilename+outputtype, dpi=dpichos, alpha=True)
        plt.close(fig)
        print("completed "+str(indef)+"/365 in "+str(year))


# stitching image into animation

In [ ]:
import moviepy.editor as mpy
import os
import glob

pngfiles=glob.glob('./*png')
output_base_name='test.mp4'
aa=[1]*len(pngfiles[0:2])
clip = mpy.ImageSequenceClip(pngfiles[0:2], durations=aa, load_images=True)
clip.write_videofile(output_base_name,audio=False,fps=24 )

#pngfiles


In [ ]:
%%HTML
from IPython.display import HTML
<video width="700" height="700" controls>
  <source src="test.mp4" type="video/mp4">
</video>